# Income Prediction: Clean the data

1. Reads raw data from a local file.
2. Transforms the data:
    - Formats column names: Change dash (-) separator to underscore (_).
    - Conflates equivalent values in the 'income' target variable to two classes: {'<=50K', '>50K'}.
    - Converts "?" categorical values to "Unknown".
3. Writes the transformed data to a local file.

In [1]:
CSV_FILE_TYPE = 'csv'
PARQUET_FILE_TYPE = 'parquet'
PARQUET_ENGINE = 'pyarrow'
DATA_FILE_TYPE = PARQUET_FILE_TYPE  # or CSV_FILE_TYPE
RAW_DATASET_PATH = f'../data/adult_income_raw.{DATA_FILE_TYPE}'
CLEANED_DATASET_PATH = f'../data/adult_income_cleaned.{DATA_FILE_TYPE}'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Reads raw data from a local file.
if DATA_FILE_TYPE == PARQUET_FILE_TYPE:
    df = pd.read_parquet(
        RAW_DATASET_PATH,
        engine=PARQUET_ENGINE,
    )
elif DATA_FILE_TYPE == CSV_FILE_TYPE:
    df = pd.read_csv(RAW_DATASET_PATH)
else:
    raise Exception(f"Unexpected {DATA_FILE_TYPE=}. Use one of ['{PARQUET_FILE_TYPE}', '{CSV_FILE_TYPE}'] instead.")
print(f"Dataframe loaded from: {RAW_DATASET_PATH}")

Dataframe loaded from: ../data/adult_income_raw.parquet


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


## Data transformations

In [5]:
# Rename columns
df.rename(
    columns={
        'fnlwgt': 'final_weight',
        'education-num': 'education_level',
        'marital-status': 'marital_status',
        'capital-gain': 'capital_gain',
        'capital-loss': 'capital_loss',
        'hours-per-week': 'hours_per_week',
        'native-country': 'native_country',
    }, 
    inplace=True
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        47879 non-null  object
 2   final_weight     48842 non-null  int64 
 3   education        48842 non-null  object
 4   education_level  48842 non-null  int64 
 5   marital_status   48842 non-null  object
 6   occupation       47876 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   sex              48842 non-null  object
 10  capital_gain     48842 non-null  int64 
 11  capital_loss     48842 non-null  int64 
 12  hours_per_week   48842 non-null  int64 
 13  native_country   48568 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [6]:
# Values before the 'income' column is made consistent.
df['income'].value_counts()

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

In [7]:
# Standardizes the values in the target column.
df['income'] = df['income'].apply(lambda x: '<=50K' if x == '<=50K.' else '>50K' if x == '>50K.' else x)
df['income'].value_counts()

income
<=50K    37155
>50K     11687
Name: count, dtype: int64

In [8]:
# Converts "?" value to "Unknown".
df['occupation'] = df['occupation'].apply(lambda x: 'Unknown' if x == '?' else x)
df['occupation'].value_counts()

occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Unknown              1843
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64

In [9]:
# Converts "?" value to "Unknown".
df['workclass'] = df['workclass'].apply(lambda x: 'Unknown' if x == '?' else x)
df['workclass'].value_counts()

workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Unknown              1836
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

In [10]:
# Converts "?" value to "Unknown".
df['native_country'] = df['native_country'].apply(lambda x: 'Unknown' if x == '?' else x)
df['native_country'].value_counts()

native_country
United-States                 43832
Mexico                          951
Unknown                         583
Philippines                     295
Germany                         206
Puerto-Rico                     184
Canada                          182
El-Salvador                     155
India                           151
Cuba                            138
England                         127
China                           122
South                           115
Jamaica                         106
Italy                           105
Dominican-Republic              103
Japan                            92
Guatemala                        88
Poland                           87
Vietnam                          86
Columbia                         85
Haiti                            75
Portugal                         67
Taiwan                           65
Iran                             59
Greece                           49
Nicaragua                        49
Peru         

## Save clean data to a local file

In [11]:
# Saves the Dataframe of cleaned data to a local file for later analysis.
if DATA_FILE_TYPE == PARQUET_FILE_TYPE:
    df.to_parquet(
        path=CLEANED_DATASET_PATH,
        engine=PARQUET_ENGINE,
        index=False,
    )
elif DATA_FILE_TYPE == CSV_FILE_TYPE:
    df.to_csv(
        path_or_buf=CLEANED_DATASET_PATH,
        index=False,
    )
else:
    raise Exception(f"Unexpected {OUTPUT_FILE_TYPE=}. Use one of ['{PARQUET_FILE_TYPE}', '{CSV_FILE_TYPE}'].")
print(f"Data saved to: {CLEANED_DATASET_PATH}")

Data saved to: ../data/adult_income_cleaned.parquet
